### **GloVe**  
임베딩 벡터의 내적이 말뭉치 전체에서의 동시출현(co-occurrence) 확률 값이 되는 목적 함수를 갖는다.  
이를 통해 임베딩 벡터간 유사도 측정을 수월하게 하면서도 말뭉치 전체의 통계 정보를 반영할 수 있다. 

> **동시 출현 (Co-occurence)** *이란, 한 문장, 문단 또는 텍스트 단위에서 같이 출현한 단어를 가리다. 언어학적 의미에서 의미적 근접성을 가리킨다.*  




In [11]:
!pip install glove_python

     |████████████████████████████████| 263 kB 1.2 MB/s eta 0:00:01
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=702684 sha256=876853786a88d51d28caccfc94eb6f4259fd52910ea9b112a5cbdac723b7190f
  Stored in directory: /root/.cache/pip/wheels/c2/34/66/a3adc1e41bd5cfe3aa8f75e34b42ca207f8b6e8171b9a4fd61
Successfully built glove-python


In [1]:
import os
import re

In [6]:
BASE_DIR = "/data/TestDir/sample_articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [7]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

기사 본문 내용이 짧기 때문에, 모델을 학습하기에 corpus의 크기가 작다.  
아래는 수집한 기사 87건들을 통해 corpus를 구성하고, GloVe 모델을 구축하는 내용이다.

In [8]:
media_list = os.listdir(ORIGIN_PATH)

result = []
forCount = []
for media in media_list:
    media_path = os.path.join(PREPROCESSED_PATH, media)
    article_list= os.listdir(media_path)

    for article in article_list:
        reader = RawTextReader(os.path.join(media_path, article)) 
        content = list(filter(None, reader))
        forCount += [token for sent in content for token in sent.split()]
        result += [sent.split() for sent in content]

In [9]:
print("전체 token의 개수 : {len}".format(len=len(forCount)))
print("중복되지 않은 token의 개수 : {len}".format(len=len(list(set(forCount)))))

전체 token의 개수 : 19548040
중복되지 않은 token의 개수 : 65848


In [12]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)

In [14]:
glove = Glove(no_components=256, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


경찰서장과 비슷한 의미의 단어를 출력한다.

In [15]:
model_result1=glove.most_similar("경찰서장")
print(model_result1)

[('양천', 0.6925531446581766), ('대기발령', 0.5270933357081705), ('서장', 0.5150034637360484), ('해양', 0.4759323725838482)]


In [16]:
corpus.dictionary["선거"]

1661

In [17]:
glove.word_vectors[corpus.dictionary["선거"]]

array([ 2.70172903e-02,  1.67197564e-01,  1.73049951e-01, -6.23310782e-03,
       -6.02483058e-02, -1.84471898e-01,  1.31673083e-01,  1.46047676e-01,
        1.49314556e-04, -2.89789628e-01,  3.50302149e-01, -8.34658024e-02,
        8.70119231e-02, -1.99333414e-01, -4.77659310e-03,  2.55370037e-01,
       -2.72165170e-01, -3.38512198e-01,  1.19463268e-01, -2.79355976e-01,
       -3.54784739e-02,  3.75653262e-01,  5.87157234e-02,  2.16822994e-01,
        1.62379695e-01,  1.99915507e-01,  3.08079690e-01,  2.34915019e-01,
       -1.87148692e-01, -7.28886458e-01,  1.71268852e-01, -9.58869891e-02,
        1.26352108e-01, -2.96362534e-01,  1.57702211e-01, -1.03395183e-01,
        4.49418497e-01,  6.06407338e-02,  1.21100642e-01,  2.13591248e-01,
       -3.57948520e-01,  3.15358655e-02, -6.12248460e-02, -2.22062260e-02,
       -2.54632950e-01, -2.05229810e-01,  2.81409816e-01, -5.05663796e-01,
       -1.97800634e-01, -7.06107938e-03,  4.84326255e-01, -9.02174402e-02,
        2.21760891e-01, -

In [19]:
glove.save('glove-256.model')
corpus.save('corpus-256.model')